In [1]:
!git clone https://github.com/toy-f-rebellion/toy_ai.git
%cd /content/toy_ai
!pip install -r requirements.txt

Cloning into 'toy_ai'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 97 (delta 28), reused 14 (delta 3), pack-reused 0
Receiving objects: 100% (97/97), 2.51 MiB | 4.90 MiB/s, done.
Resolving deltas: 100% (28/28), done.
/content/toy_ai
Cloning into 'Modified-KoBERT'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 449 (delta 136), reused 118 (delta 108), pack-reused 275
Receiving objects: 100% (449/449), 226.32 KiB | 3.02 MiB/s, done.
Resolving deltas: 100% (227/227), done.
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-fx61d4s1/kobert-tokenizer_8037825d59ac43a0aa57db92a45d9a75
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-fx61d4s1/kobert-tokenizer_8037825d59ac43a0aa57db92a45d9a75
  Resolved http

# 1.환경설정

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## (1) 라이브러리 불러오기

In [8]:
import warnings
warnings.filterwarnings('ignore')

# Setting Library
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle

# koBERT
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

# Transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

device = torch.device('cuda')

## (2) 데이터 불러오기

In [5]:
with open('/content/toy_ai/train_data.pkl', 'rb') as f:
    train_data = pickle.load(f)
with open('/content/toy_ai/test_data.pkl', 'rb') as f:
    test_data = pickle.load(f)

## (3) KoBERT 모델 불러오기

In [9]:
# KoBERT로부터 model, vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/toy_ai/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/toy_ai/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 2.Modeling

## (1) 입력 데이터셋 토큰화

- 함수 정의

In [10]:
# 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 함수 정의
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [11]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/toy_ai/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


- Setting parameters

In [12]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1000
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

- 토큰화

In [13]:
train_data = BERTDataset(train_data, 0, 1, tok, max_len, True, False)
test_data = BERTDataset(test_data,0, 1, tok, max_len, True, False)

- torch 형식으로 반환

In [14]:
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=5)

## (2) EarlyStopping

In [15]:
# Early Stopping
class EarlyStopping:
    def __init__(self, patience=5, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float("Inf")

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...")
        torch.save(model.state_dict(), "checkpoint.pt")
        self.val_loss_min = val_loss

## (3) Modeling

- KoBERT 함수 정의

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [17]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

- Training

In [18]:
train_history=[]
test_history=[]
loss_history=[]

early_stopping = EarlyStopping(patience=5, verbose=True) # EarlyStopping 적용

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    train_history.append(train_acc / (batch_id+1))

    model.eval()
    y_pred, y_test = [], []
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

    # Early stopping 및 ModelCheckpoint 적용
    epoch_test_loss = loss.item()
    early_stopping(epoch_test_loss, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break

# 최적의 모델 불러오기
model.load_state_dict(torch.load("checkpoint.pt"))

  0%|          | 0/243 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.0302786827087402 train acc 0.15625
epoch 1 batch id 201 loss 1.9005939960479736 train acc 0.14412313432835822
epoch 1 train acc 0.1509539842873176


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 1 test acc 0.18812939110070256
Validation loss decreased (inf --> 2.063841). Saving model...


  0%|          | 0/243 [00:00<?, ?it/s]

...

  0%|          | 0/243 [00:00<?, ?it/s]

epoch 44 batch id 1 loss 0.0037550036795437336 train acc 1.0
epoch 44 batch id 201 loss 0.08241549879312515 train acc 0.990282960199005
epoch 44 train acc 0.9904835390946503


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 44 test acc 0.9158591920374707
EarlyStopping counter: 5 out of 5
Early stopping


<All keys matched successfully>

# 3. Model Save

In [19]:
torch.save(model, '/content/toy_ai/model.pth')